<font size="10" color="purple"><left><b><i><u>Exploratory Data Analysis #2</u></i></b></left></font>

<font size="4"><li><left><i>Instanciamos clase EDA con los métodos necesarios</i></left></font>


In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
#import pandas_profiling as pdpf
from IPython.display import HTML
#import sweetviz as sv

class EDA:
    def read_csv(self, file_path):
        """
        Lee un archivo CSV y devuelve un dataframe.

        Args:
            file_path (str): Ruta al archivo CSV a leer.

        Returns:
            pd.DataFrame: El dataframe con los datos del archivo CSV.
        """
        dataframe = pd.read_csv(file_path,encoding='UTF-8')
        return dataframe

    def get_columnas(self, dataframe):
        """
        Obtiene una lista de las columnas de un dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe del cual se desean obtener las columnas.

        Returns:
            list: Lista de las columnas del dataframe.
        """
        columnas = dataframe.columns.tolist()
        return columnas

    def set_columns_dtype_category(self, dataframe, columnas):
        """
        Asigna el tipo de dato "category" a una lista de columnas en un dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe al que se desea asignar los tipos de dato.
            columnas (list): La lista de columnas a las que se desea asignar el tipo de dato "category".
        """
        dataframe[columnas] = dataframe[columnas].astype('category')

    def set_columns_dtype_int(self, dataframe, columnas):
        """
        Asigna el tipo de dato "int" a una lista de columnas en un dataframe.
        Los valores nulos en las columnas seleccionadas se reemplazan por 0.

        Args:
            dataframe (pd.DataFrame): El dataframe al que se desea asignar los tipos de dato.
            columnas (list): La lista de columnas a las que se desea asignar el tipo de dato "int".
        """
        dataframe[columnas] = dataframe[columnas].astype(int).fillna(0)

    def set_columns_dtype_float(self, dataframe, columnas):
        """
        Asigna el tipo de dato "float" a una lista de columnas en un dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe al que se desea asignar los tipos de dato.
            columnas (list): La lista de columnas a las que se desea asignar el tipo de dato "float".
        """
        dataframe[columnas] = dataframe[columnas].astype(float)

    def set_columns_dtype_datetime(self, dataframe, columnas, formato='%Y-%m-%d'):
        """
        Asigna el tipo de dato "datetime" a una lista de columnas en un dataframe, con el formato especificado.

        Args:
            dataframe (pd.DataFrame): El dataframe al que se desea asignar los tipos de dato.
            columnas (list): La lista de columnas a las que se desea asignar el tipo de dato "datetime".
            formato (str): El formato de fecha a utilizar (por defecto: "%Y-%m-%d").
        """
        dataframe[columnas] = pd.to_datetime(dataframe[columnas], format=formato)

    def rename_columns(self, dataframe, column_mapping):
        """
        Modifica los nombres de las columnas de un dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe al que se le van a modificar los nombres de las columnas.
            column_mapping (dict): Un diccionario que contiene el mapeo de los nombres actuales de las columnas a los nuevos nombres.

        Returns:
            pd.DataFrame: El dataframe con los nombres de columnas modificados.
        """
        dataframe.rename(columns=column_mapping, inplace=True)
        return dataframe

    def get_duplicate_rows(self, dataframe, column):
        """
        Devuelve las filas del dataframe que tienen valores repetidos en una columna específica,
        junto con la cantidad de repeticiones y el porcentaje respecto al total de líneas.

        Args:
            dataframe (pd.DataFrame): El dataframe a analizar.
            column (str): El nombre de la columna en la cual buscar repeticiones.

        Returns:
            pd.DataFrame: Un dataframe con las filas que se repiten, la cantidad de repeticiones
                        y el porcentaje respecto al total de líneas.
        """
        duplicated_rows = dataframe[dataframe.duplicated(subset=column, keep=False)]
        duplicated_counts = duplicated_rows[column].value_counts().reset_index()
        duplicated_counts.columns = [column, 'Cantidad']
        duplicated_counts['Porcentaje'] = duplicated_counts['Cantidad'] / len(dataframe) * 100
        return duplicated_counts
    
    def get_duplicate_rows_2(self, dataframe, column):
        """
        Devuelve las filas del dataframe que tienen valores repetidos en una columna específica.
        Incluye las otras columnas del dataframe, la cantidad y el porcentaje de filas duplicadas.
        Ordena el resultado por los valores de la columna especificada.

        Args:
            dataframe (pd.DataFrame): El dataframe a analizar.
            column (str): El nombre de la columna en la que se buscarán valores duplicados.

        Returns:
            pd.DataFrame: El dataframe con las filas duplicadas, incluyendo las otras columnas,
                        la cantidad y el porcentaje de filas duplicadas. Ordenado por la columna especificada.
        """
        duplicates = dataframe[dataframe.duplicated(subset=column, keep=False)]
        duplicate_counts = duplicates[column].value_counts()
        duplicate_percentages = duplicate_counts / len(dataframe) * 100

        duplicates['Duplicate Count'] = duplicates[column].map(duplicate_counts)
        duplicates['Duplicate Percentage'] = duplicates[column].map(duplicate_percentages)

        duplicates = duplicates.sort_values(by=column)

        return duplicates

    def strip_column_values(self, dataframe, column_names):
        """
        Elimina los espacios en blanco al inicio y al final de los valores de las filas de las columnas especificadas.

        Args:
            dataframe (pd.DataFrame): El dataframe en el cual se van a limpiar los valores de las columnas.
            column_names (list): Una lista de nombres de columnas a las cuales se les va a aplicar el método .strip().

        Returns:
            pd.DataFrame: El dataframe con los valores de las filas de las columnas especificadas limpios de espacios en blanco.
        """
        for column in column_names:
            dataframe[column] = dataframe[column].str.strip()
        return dataframe

    def sort_dataframe(self, dataframe, column, order='a'):
        """
        Ordena un dataframe por una columna especificada.

        Args:
            dataframe (pd.DataFrame): El dataframe a ordenar.
            column (str): El nombre de la columna por la cual ordenar el dataframe.
            order (str, opcional): El orden de clasificación. 'a' para ascendente (predeterminado),
                'd' para descendente.

        Returns:
            pd.DataFrame: El dataframe ordenado por la columna especificada.
        """
        if order == 'd':
            dataframe = dataframe.sort_values(column, ascending=False)
        else:
            dataframe = dataframe.sort_values(column)

        return dataframe

    def sort_and_replace(self, dataframe, column1, column2):
        """
        Ordena un dataframe de menor a mayor según los valores de una columna y reemplaza los valores de otra columna por enteros.

        Args:
            dataframe (pd.DataFrame): El dataframe a procesar.
            column1 (str): El nombre de la primera columna según la cual se va a ordenar el dataframe.
            column2 (str): El nombre de la segunda columna cuyos valores se reemplazarán por enteros.

        Returns:
            pd.DataFrame: El dataframe ordenado y con los valores de la segunda columna reemplazados por enteros.
        """
        # Ordenar dataframe por la columna 1 de menor a mayor
        dataframe = dataframe.sort_values(by=column1)

        # Reemplazar los valores de la columna 2 por enteros
        dataframe[column2] = range(len(dataframe))

        return dataframe
        
    def convert_date_format(self, dataframe, columns):
        """
        Convierte el formato de fecha de columnas en un dataframe de 'MMM DD, YYYY' a 'YYYY-MM-DD'
        y asigna el dtype datetime a las columnas especificadas.

        Args:
            dataframe (pd.DataFrame): El dataframe a procesar.
            columns (list): Una lista de nombres de columnas a convertir y asignar el dtype datetime.

        Returns:
            pd.DataFrame: El dataframe con los formatos de fecha modificados y dtype datetime asignado.
        """
        for column in columns:
            dataframe[column] = pd.to_datetime(dataframe[column], format='%b %d, %Y').dt.strftime('%Y-%m-%d')
            dataframe[column] = pd.to_datetime(dataframe[column])
            
        return dataframe

    def remove_prefix(self, dataframe, column, prefix):
        """
        Elimina un prefijo de las filas de una columna en un dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe a procesar.
            column (str): El nombre de la columna en la que se eliminará el prefijo.
            prefix (str): El prefijo a eliminar de las filas de la columna.

        Returns:
            pd.DataFrame: El dataframe con el prefijo eliminado de las filas de la columna especificada.
        """
        dataframe[column] = dataframe[column].str.lstrip(prefix)
        return dataframe
    
    def transform_values(self, dataframe, columns, symbols):
        """
        Transforma los valores de las columnas especificadas en el dataframe, eliminando símbolos, realizando un strip y convirtiendo a minúscula.

        Args:
            dataframe (pd.DataFrame): El dataframe a transformar.
            columns (list): La lista de nombres de columnas a transformar.
            symbols (list): La lista de símbolos a eliminar.

        Returns:
            pd.DataFrame: El dataframe con los valores transformados.
        """
        for column in columns:
            for symbol in symbols:
                dataframe[column] = dataframe[column].str.replace(symbol, '')
            dataframe[column] = dataframe[column].str.strip().str.lower()

        return dataframe

    def replace_na_values_obj(self, dataframe, columns, replacement):
        """
        Reemplaza los valores NaN en las columnas especificadas del dataframe con un valor de reemplazo.

        Args:
            dataframe (pd.DataFrame): El dataframe en el que se reemplazarán los valores NaN.
            columns (list): La lista de nombres de columnas en las que se realizará el reemplazo.
            replacement (str): El valor de reemplazo para los valores NaN.

        Returns:
            pd.DataFrame: El dataframe con los valores NaN reemplazados.
        """
        dataframe[columns] = dataframe[columns].fillna(replacement)
        return dataframe

    def convert_values_to_int(self,dataframe, columns, str_to_remove):
        """
        Convierte los valores de las columnas seleccionadas del dataframe en enteros.

        Los valores de las columnas seleccionadas que contengan el str indicado se eliminan.
        Los valores restantes se convierten a enteros.

        Args:
            dataframe (pd.DataFrame): El dataframe original.
            columns (list): Lista de nombres de columnas a modificar.
            str_to_remove (str): El str a eliminar de los valores de las columnas.

        Returns:
            pd.DataFrame: El dataframe modificado con los valores convertidos a enteros.
        """
        for column in columns:
            dataframe[column] = dataframe[column].str.replace(str_to_remove, '')
            dataframe[column] = pd.to_numeric(dataframe[column], errors='coerce').astype(pd.Int64Dtype())

        return dataframe

    def convert_values_to_float(self,dataframe, columns, str_to_remove):
        """
        Convierte los valores de las columnas seleccionadas del dataframe en flotantes.

        Los valores de las columnas seleccionadas que contengan el str indicado se eliminan.
        Los valores restantes se convierten a flotantes.

        Args:
            dataframe (pd.DataFrame): El dataframe original.
            columns (list): Lista de nombres de columnas a modificar.
            str_to_remove (str): El str a eliminar de los valores de las columnas.

        Returns:
            pd.DataFrame: El dataframe modificado con los valores convertidos a flotantes.
        """
        for column in columns:
            dataframe[column] = dataframe[column].str.replace(str_to_remove, '')
            dataframe[column] = pd.to_numeric(dataframe[column], errors='coerce').astype(float())

        return dataframe

    def plot_histogram(self, dataframe, column):
        """
        Crea un histograma utilizando countplot para una columna dada en un dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe del que se extraen los datos.
            column (str): El nombre de la columna para el histograma.
        """
        sns.countplot(data=dataframe, x=column)
        plt.title(f'Histograma de {column}')
        plt.xlabel(column)
        plt.ylabel('Frecuencia')
        
        # Calcular los porcentajes de cada categoría
        total_count = len(dataframe)
        ax = plt.gca()
        for p in ax.patches:
            percentage = '{:.1f}%'.format(100 * p.get_height() / total_count)
            x = p.get_x() + p.get_width() / 2
            y = p.get_height()
            ax.annotate(percentage, (x, y), ha='center', va='bottom')
        
        # Formatear el eje y para mostrar porcentajes
        ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y/total_count)))
        
        # Mostrar la imagen en el entorno de ejecución de Jupyter Notebook
        plt.show()
    
    def drop_columns(self, dataframe, column):
        """
        Elimina las columnas especificadas del DataFrame.

        Args:
            column_names (list): Lista de nombres de columnas a eliminar.
        """
        for column_name in column:
            if column_name in dataframe.columns:
                dataframe = dataframe.drop(column_name, axis=1)
            else:
                print(f"La columna '{column_name}' no existe en el DataFrame. Se omitirá.")
        return dataframe

    def replace_null_values(self, dataframe):
        """
        Reemplaza los valores nulos en un dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe al que se desea reemplazar los valores nulos.

        Returns:
            pd.DataFrame: El dataframe modificado con los valores nulos reemplazados.
        """
        dataframe = dataframe.fillna({
            column: '' if dtype == 'object' else 0
            for column, dtype in dataframe.dtypes.items()
        })
        return dataframe
    
    def transform_columns_to_int(self,dataframe, columns):
        """
        Transforma las columnas indicadas de un dataframe a tipo entero (int) y reemplaza los valores nulos por 0.

        Args:
            dataframe (pd.DataFrame): El dataframe al que se desea aplicar la transformación.
            columns (list): La lista de nombres de columna a transformar.

        Returns:
            pd.DataFrame: El dataframe modificado con las columnas transformadas a tipo entero y los valores nulos reemplazados por 0.
        """
        for column in columns:
            dataframe[column] = dataframe[column].fillna(0).astype(int)
        return dataframe

    def convert_column_to_int(self, dataframe, column_name):
        """
        Convierte los valores de una columna en un dataframe a enteros, después de eliminar los caracteres no numéricos.

        Args:
            dataframe (pd.DataFrame): El dataframe que contiene la columna.
            column_name (str): El nombre de la columna a convertir.

        Returns:
            pd.DataFrame: El dataframe modificado con los valores convertidos a enteros.
        """
        dataframe[column_name] = dataframe[column_name].str.replace(r'\D+', '', regex=True)
        dataframe[column_name] = pd.to_numeric(dataframe[column_name], errors='coerce').astype(pd.Int64Dtype())

        return dataframe

    def replace_values(self, dataframe, column_name, replace_dict):
        """
        Reemplaza los valores de una columna en un dataframe utilizando un diccionario de búsqueda y reemplazo.

        Args:
            dataframe (pd.DataFrame): El dataframe en el que se desea reemplazar los valores.
            column_name (str): El nombre de la columna en la que se desea realizar el reemplazo.
            replace_dict (dict): Un diccionario que contiene los caracteres a buscar como claves y los caracteres
                                de reemplazo como valores.

        Returns:
            pd.DataFrame: El dataframe con los valores de la columna reemplazados.
        """
        dataframe[column_name] = dataframe[column_name].replace(replace_dict, regex=True)
        return dataframe

    def extract_average_value(self, dataframe, column):
        """
        Extrae el valor promedio de un rango numérico en una columna del dataframe.

        Args:
            dataframe (pd.DataFrame): El dataframe que contiene la columna.
            column (str): El nombre de la columna a transformar.

        Returns:
            pd.DataFrame: El dataframe con los valores modificados en la columna seleccionada.
        """
        # Expresión regular para buscar el rango numérico
        pattern = r"(\d+)[^\d]+(\d+)"
        
        # Función para calcular el promedio de los valores en el rango
        def calculate_average(match):
            num1 = int(match.group(1))
            num2 = int(match.group(2))
            average = (num1 + num2) / 2
            return str(average)
        
        # Aplicar la transformación a la columna
        dataframe[column] = dataframe[column].apply(lambda x: re.sub(pattern, calculate_average, x) if pd.notnull(x) else x)
        
        return dataframe
    
    def reorder_columns(self,dataframe, column_mapping):
        """
        Reordena las columnas de un dataframe de acuerdo a un diccionario de mapeo.

        Args:
            dataframe (pd.DataFrame): El dataframe al que se le reordenarán las columnas.
            column_mapping (dict): El diccionario de mapeo que especifica el nuevo orden de las columnas.

        Returns:
            pd.DataFrame: El dataframe con las columnas reordenadas.
        """
        return dataframe[column_mapping.keys()].rename(columns=column_mapping)


# Instancia de la clase EDA
eda = EDA()

<font size="4"><li><left><i>Exportamos Coursera_reviews.csv a dataframe</i></left></font>

In [2]:
# Path de archivos CSV
path = '/Users/negro/Library/CloudStorage/OneDrive-Personal/Documentos/00 Fran/01 - Personales/02-Learn/0. Data Science/0. Data Science/2_projects/e_MOOCs/2_Repo/MOOCs_BA/rawDataSets/Coursera_reviews.csv'

# Leer y almacenar los archivos CSV en dataframes
Coursera_reviews = eda.read_csv(path)


<font size="4"><li><left><i>Obtenemos información del dataframe</i></left></font>

In [3]:
Coursera_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454711 entries, 0 to 1454710
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   reviews       1454571 non-null  object
 1   reviewers     1454711 non-null  object
 2   date_reviews  1454711 non-null  object
 3   rating        1454711 non-null  int64 
 4   course_id     1454711 non-null  object
dtypes: int64(1), object(4)
memory usage: 55.5+ MB


In [ ]:
# # Generar el perfil del dataframe
# profile = Coursera_reviews.profile_report(title='Informe de Pandas-Profiling', explorative=True)
# # Guardar el informe en un archivo HTML
# profile.to_file('/Users/negro/Library/CloudStorage/OneDrive-Personal/Documentos/00 Fran/01 - Personales/02-Learn/0. Data Science/0. Data Science/2_projects/e_MOOCs/2_Repo/MOOCs_BA/reports/Coursera_reviews.html')
# # Cargar y mostrar el informe HTML
# report_html = open('/Users/negro/Library/CloudStorage/OneDrive-Personal/Documentos/00 Fran/01 - Personales/02-Learn/0. Data Science/0. Data Science/2_projects/e_MOOCs/2_Repo/MOOCs_BA/reports/Coursera_reviews.html', 'r').read()
# display(HTML(report_html))


<font size="4"><li><left><i>No se identifican valores null en este dataset.</i></left></font><br>

<font size="4"><li><left><i>Identificamos duplicados:</i></left></font><br>

In [19]:
num_duplicados = len(Coursera_reviews[Coursera_reviews.duplicated()])
porcentaje_duplicados = (num_duplicados/len(Coursera_reviews))*100
print(porcentaje_duplicados,'%')

64.25771166919066 %


In [13]:
ejemplos_duplicados = Coursera_reviews[Coursera_reviews.duplicated()]
ejemplos_duplicados_ordenados = ejemplos_duplicados.sort_values(by='reviews')
ejemplos_duplicados_ordenados.head(10)

,reviews,reviewers,date_reviews,rating,course_id
842146,"\tAs a general feedback, first I would ask you...",By Carlos O,"Sep 15, 2020",5,astro
842171,"\tAs a general feedback, first I would ask you...",By Carlos O,"Sep 15, 2020",5,astro
1238334,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAn\nexcell...,By Alan K,"May 31, 2020",5,systems-engineering
1238359,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAn\nexcell...,By Alan K,"May 31, 2020",5,systems-engineering
385952,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nEste\nes u...,By Jesus M,"Feb 25, 2020",5,pap
385927,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nEste\nes u...,By Jesus M,"Feb 25, 2020",5,pap
962888,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nExcellent\...,By William B e,"Jun 12, 2019",5,wharton-fintech-overview-payments-regulations
962913,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nExcellent\...,By William B e,"Jun 12, 2019",5,wharton-fintech-overview-payments-regulations
1353420,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHa sido\nd...,By ELIZABETH S F,"Jul 18, 2020",5,aulaconstructivista
1353445,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHa sido\nd...,By ELIZABETH S F,"Jul 18, 2020",5,aulaconstructivista


<font size="4"><li><left><i>Eliminamos filas duplicadas:</i></left></font><br>

In [20]:
Coursera_reviews = Coursera_reviews.drop_duplicates()
Coursera_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 519947 entries, 0 to 1454644
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   reviews       519891 non-null  object
 1   reviewers     519947 non-null  object
 2   date_reviews  519947 non-null  object
 3   rating        519947 non-null  int64 
 4   course_id     519947 non-null  object
dtypes: int64(1), object(4)
memory usage: 23.8+ MB


<font size="4"><li><left><i>Clasificamos variables y renombramos columnas:</i></left></font>

In [21]:
# Obtener cabecera del dataframe
Coursera_reviews.head()

,reviews,reviewers,date_reviews,rating,course_id
0,"Pretty dry, but I was able to pass with just t...",By Robert S,"Feb 12, 2020",4,google-cbrs-cpi-training
1,would be a better experience if the video and ...,By Gabriel E R,"Sep 28, 2020",4,google-cbrs-cpi-training
2,Information was perfect! The program itself wa...,By Jacob D,"Apr 08, 2020",4,google-cbrs-cpi-training
3,A few grammatical mistakes on test made me do ...,By Dale B,"Feb 24, 2020",4,google-cbrs-cpi-training
4,Excellent course and the training provided was...,By Sean G,"Jun 18, 2020",4,google-cbrs-cpi-training


In [22]:
# Obtener lista de columnas
columnas = eda.get_columnas(Coursera_reviews)
print(columnas)

['reviews', 'reviewers', 'date_reviews', 'rating', 'course_id']


<font size="4"><li><left><i>Definimos variables cualitativas:</i></left></font><br>
<br>
<font size="4"><left><i>- Ordinales: `rating`</i></left></font><br>
<br>
<font size="4"><left><i>- Nominales: `reviews` y `reviewers`</i></left></font><br>
<br>
<font size="4"><li><left><i>Definimos variables cuantitativas:</i></left></font><br>
<br>
<font size="4"><left><i>- No se distinguen en este dataset</i></left></font><br>
<br>
<font size="4"><li><left><i>Posible PK:</i></left></font><br>
<br>
<font size="4"><left><i>- `course_id`</i></left></font><br>


In [23]:
# Crear una lista con las columnas que objetos con formato daytime, asiganar en formato 'MMM DD, YYYY'
columnas_fecha = ['date_reviews']
Coursera_reviews = eda.convert_date_format(Coursera_reviews, columnas_fecha)


In [24]:
# Obtener cabecera del dataframe
Coursera_reviews.head()

,reviews,reviewers,date_reviews,rating,course_id
0,"Pretty dry, but I was able to pass with just t...",By Robert S,2020-02-12,4,google-cbrs-cpi-training
1,would be a better experience if the video and ...,By Gabriel E R,2020-09-28,4,google-cbrs-cpi-training
2,Information was perfect! The program itself wa...,By Jacob D,2020-04-08,4,google-cbrs-cpi-training
3,A few grammatical mistakes on test made me do ...,By Dale B,2020-02-24,4,google-cbrs-cpi-training
4,Excellent course and the training provided was...,By Sean G,2020-06-18,4,google-cbrs-cpi-training


In [25]:
Coursera_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 519947 entries, 0 to 1454644
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   reviews       519891 non-null  object        
 1   reviewers     519947 non-null  object        
 2   date_reviews  519947 non-null  datetime64[ns]
 3   rating        519947 non-null  int64         
 4   course_id     519947 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 23.8+ MB


<font size="4" color="white"><li><left><i>Eliminamos espacios antes/despues:</i></left></font>

In [26]:
# Eliminar espacios alrededor de str
# Definir una lista de nombres de columnas a limpiar
columnas_a_limpiar = ['reviews', 'reviewers','course_id']

# Llamar al método strip_column_values
Coursera_reviews = eda.strip_column_values(Coursera_reviews, columnas_a_limpiar)

<font size="4" color="white"><li><left><i>Eliminamos el prefijo `By `de los valores de la columna `reviewers`:</i></left></font>

In [27]:
Coursera_reviews = eda.remove_prefix(Coursera_reviews, 'reviewers', 'By ')
Coursera_reviews.head()

,reviews,reviewers,date_reviews,rating,course_id
0,"Pretty dry, but I was able to pass with just t...",Robert S,2020-02-12,4,google-cbrs-cpi-training
1,would be a better experience if the video and ...,Gabriel E R,2020-09-28,4,google-cbrs-cpi-training
2,Information was perfect! The program itself wa...,Jacob D,2020-04-08,4,google-cbrs-cpi-training
3,A few grammatical mistakes on test made me do ...,Dale B,2020-02-24,4,google-cbrs-cpi-training
4,Excellent course and the training provided was...,Sean G,2020-06-18,4,google-cbrs-cpi-training


 <font size="4" color="white"><li><left><i>Analizamos duplicados:</i></left></font>

In [28]:
# Traer filas con duplicados en 'reviews'
filas_duplicadas = eda.get_duplicate_rows(Coursera_reviews, 'reviews')
filas_duplicadas.head(10)

,reviews,Cantidad,Porcentaje
0,good,3397,0.653336
1,Good,2256,0.433890
2,Excellent,1654,0.318109
3,Great course!,1014,0.195020
4,great,867,0.166748
5,Excellent course,865,0.166363
6,Great course,825,0.158670
7,excellent,821,0.157901
8,nice,810,0.155785
9,very good,803,0.154439


<font size="4"><li><left><i>La reseña que más se repite es `good`, pero se nota que hay diferencias en valores que consideraremos iguales.</i></left></font>
<br>
<br>
<font size="4"><li><left><i>Armonizamos la fila para agrupar de mejor manera:</i></left></font>

In [37]:
# Reemplzar caracteres irrelevantes y homogenizar a minúsculas
columns_to_transform = ['reviews']
symbols_to_remove = ['!', 'course', 'curso', 'Curso', 'Course']

Coursera_reviews = eda.transform_values(Coursera_reviews, columns_to_transform, symbols_to_remove)

# Traer filas con duplicados en 'resenia'
filas_duplicadas = eda.get_duplicate_rows(Coursera_reviews, 'reviews')
filas_duplicadas.head(20)

,reviews,Cantidad,Porcentaje
0,excellent,10182,1.958277
1,good,7852,1.510154
2,great,6780,1.303979
3,very good,2709,0.521015
4,nice,2464,0.473894
5,awesome,1934,0.371961
6,amazing,1463,0.281375
7,thank you,779,0.149823
8,very useful,778,0.149631
9,very nice,625,0.120205


<font size="4"><li><left><i>Dada la relevancia de `excellent` y `great`, modificamos valores similares que representan el mismo `reviews`:</i></left></font>

In [36]:
Coursera_reviews['reviews'] = Coursera_reviews['reviews'].str.replace('excelente', 'excellent')
Coursera_reviews['reviews'] = Coursera_reviews['reviews'].str.replace('excelent', 'excellent')
Coursera_reviews['reviews'] = Coursera_reviews['reviews'].str.replace('excellent .', 'excellent')
Coursera_reviews['reviews'] = Coursera_reviews['reviews'].str.replace('great .', 'great')

<font size="4"><li><left><i>La distribución de `reviews` muestra una alta disperción, con algunos valores relevantes como  `excellent`, `good` y `great`.</i></left></font>

In [38]:
# Traer filas con duplicados en 'reviewers'
filas_duplicadas = eda.get_duplicate_rows(Coursera_reviews, 'reviewers')
filas_duplicadas.head(10)

,reviewers,Cantidad,Porcentaje
0,Deleted A,1930,0.371192
1,Muhammad A,234,0.045005
2,David M,153,0.029426
3,Michael S,152,0.029234
4,Abhishek S,152,0.029234
5,Michael M,140,0.026926
6,David S,133,0.025580
7,Michael B,132,0.025387
8,David B,130,0.025003
9,Aditya S,129,0.024810


<font size="4"><li><left><i>El `reviewers` que más se repite es `Deleted A` y representa un `0,37%` del total, esperada alta disperción en este atributo. No se tiene suficiente información para determinar si esto es un error o efectivamente el nombre `Deleted` es real. </i></left></font>

In [39]:
# Traer filas con duplicados en 'date_reviews'
filas_duplicadas = eda.get_duplicate_rows(Coursera_reviews, 'date_reviews')
filas_duplicadas.head(5)


,date_reviews,Cantidad,Porcentaje
0,2020-05-18,1975,0.379846
1,2020-05-25,1968,0.378500
2,2020-05-31,1951,0.375231
3,2020-06-01,1945,0.374077
4,2020-05-11,1911,0.367537


<font size="4"><li><left><i>La distribución de `date_reviews` muestra una alta disperción.</i></left></font>

In [40]:
# Traer filas con duplicados en 'calificacion'
filas_duplicadas = eda.get_duplicate_rows(Coursera_reviews, 'rating')
filas_duplicadas.head(20)

,rating,Cantidad,Porcentaje
0,5,408118,78.492231
1,4,81401,15.655634
2,3,17733,3.410540
3,1,6625,1.274168
4,2,6070,1.167427


<font size="4"><li><left><i>Más de un `78%` de `calificacion` es igual a `5`.</i></left></font>

In [41]:
# Traer filas con duplicados en 'course_id'
filas_duplicadas = eda.get_duplicate_rows(Coursera_reviews, 'course_id')
filas_duplicadas.head(20)

,course_id,Cantidad,Porcentaje
0,python,15226,2.928375
1,machine-learning,12677,2.438133
2,neural-networks-deep-learning,12292,2.364087
3,technical-support-fundamentals,12055,2.318506
4,learning-how-to-learn,11871,2.283117
5,python-data,11422,2.196762
6,the-science-of-well-being,8199,1.576891
7,what-is-datascience,7397,1.422645
8,ai-for-everyone,7386,1.420529
9,excel-essentials,7377,1.418798


<font size="4"><li><left><i>El `curse_id` con más `reviews` es `python` con `2,9%`, bastante similar a todo el top5.</i></left></font>

<font size="4"><li><left><i>Creamos la tabla `resultado` con los valores agregados para cada `course_id`, entregando el `rating` promedio, la sumatoria de `reviewers` y la fecha mínima que tomaremos como `published_timestamp`</i></left></font>

In [42]:
resultado = Coursera_reviews.groupby('course_id').agg({'course_id':'first','rating': 'mean', 'reviewers': 'count', 'date_reviews':'min'})
resultado.head()

,course_id,rating,reviewers,date_reviews
course_id,,,,
aboriginal-education,aboriginal-education,4.552239,67,2017-03-22
access-control-sscp,access-control-sscp,4.466667,45,2018-04-19
accounting-analytics,accounting-analytics,4.395299,468,2016-01-05
accounting-data-analytics-python,accounting-data-analytics-python,3.500000,8,2020-01-11
actualizacion-manejo-diabetes-tipo-2,actualizacion-manejo-diabetes-tipo-2,4.845850,253,2019-02-05


In [28]:
#Exportamos dataframe a csv
# Definimos path
path_out_csv = '/Users/negro/Library/CloudStorage/OneDrive-Personal/Documentos/00 Fran/01 - Personales/02-Learn/0. Data Science/0. Data Science/2_projects/e_MOOCs/2_Repo/MOOCs_BA/rawDataSets/Coursera_reviews_agg.csv'

resultado.to_csv(path_out_csv, index=False, encoding='UTF-8', decimal='.')
